#### 1. 필요한 라이브러리 및 패키지 임포트

In [ ]:
import pandas as pd
import numpy as np

# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
from bs4 import BeautifulSoup as bs
import requests

import re
import urllib.request

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium .webdriver.chrome.service import Service
import random

#### 2. 나라장터 크롤링 함수

##### 2.1 최대 페이지 탐색

In [ ]:
def get_max_page(driver):
    try:
        # 페이지 네비게이션의 페이지 번호 리스트를 찾음
        page_elements = driver.find_elements(By.CSS_SELECTOR, ".pagination .page a")

        try:
            # 마지막 페이지 번호 확인 (페이지 네비게이션 마지막 부분에 있는 'page_last' 클래스를 기준으로 최대 페이지 구함)
            last_page_element = driver.find_element(By.CSS_SELECTOR, ".pagination .page_last")
            max_page = int(last_page_element.get_attribute("href").split("'")[1])
           
            return max_page
        
        except NoSuchElementException:
            # 'page_last' 요소가 없는 경우 page_elements의 길이만큼 페이지를 반환
            return len(page_elements)

    except Exception as e:
        print(f"페이지 탐색 중 오류 발생: {e}")
        return 1

##### 2.2 개찰공고 식별번호 추출

In [ ]:
def nara_crawler(search_word, start_date, end_date):
    search_query = search_word
    euc_kr_encoded = search_query.encode('euc-kr') # 문자열을 EUC-KR로 인코딩
    query = urllib.parse.quote(euc_kr_encoded) # URL 인코딩

    # 인코딩 한 문자열을 다시 검색어로 디코딩
      #decoded_bytes = urllib.parse.unquote_to_bytes(query)
      #decoded_str = decoded_bytes.decode('euc-kr')

    first_url = 'https://www.g2b.go.kr:8340/search.do?kwd=' + query + f'&category=GC&subCategory=ALL&detailSearch=true&reSrchFlag=false&pageNum=1&sort=ODD&srchFd=ALL&date=&startDate={start_date}&endDate={end_date}'
    driver = webdriver.Chrome()
    driver.get(first_url)
    time.sleep(random.uniform(1, 2))

    # 팝업창이 뜨면 닫는 동작 생성
    main = driver.window_handles
    for i in main:
        if i != main[0]:
            driver.switch_to.window(i)
            driver.close() # 창 닫기

    # 개찰결과 검색 결과가 0건인 경우 예외처리
    result_element = driver.find_element(By.CSS_SELECTOR, 'h3.tit')
    if result_element: # 검색 결과 개수 추출
        result_text = result_element.text
        match = re.search(r'\((\d+)건\)', result_text)

        if match:
            result_count = int(match.group(1))
            
            if result_count == 0: # 개찰결과가 0건인 경우 
                print("⚠️검색 결과가 없습니다.")
                driver.quit()  # 드라이버 종료
                return None 

    # 검색 결과의 최대 페이지 수 탐색
    max_page = get_max_page(driver)

    bidno = [] # 식별번호 저장 리스트

    for i in range(1, max_page+1):
        url = 'https://www.g2b.go.kr:8340/search.do?kwd=' + query + f'&category=GC&subCategory=ALL&detailSearch=true&reSrchFlag=false&pageNum={i}&sort=ODD&srchFd=ALL&date=&startDate={start_date}&endDate={end_date}'

        # 헤더 변경으로 크롤링 차단 우회
        HEADERS = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
            "Accept-Language": "ko-KR,ko;q=0.9",
            "Accept-Encoding": "gzip, deflate, br",
            "Connection": "keep-alive",
            "Upgrade-Insecure-Requests": "1",
            "TE": "Trailers",
            "DNT": "1",
        }

        driver=webdriver.Chrome()
        driver.get(url)
        time.sleep(random.uniform(1, 2))

        # 팝업창이 뜨면 닫는 동작 생성
        main = driver.window_handles
        for i in main:
            if i != main[0]:
                driver.switch_to.window(i)
                driver.close() # 창 닫기

        # 한 페이지의 결과 리스트
        ul_elements = driver.find_elements(By.XPATH, '//*[@id="contents"]/div[1]/ul')
        pattern = re.compile(r'\[([^\]]+)\]')

        # 리스트에서 각각의 결과 요소 추출
        for ul in ul_elements:
            li_elements = ul.find_elements(By.TAG_NAME, 'li')

            for li in li_elements:
                text = li.text
                matches = pattern.findall(text)

                for match in matches:
                    split_values = match.split('-')
                    
                    if len(split_values) == 2:
                        value1, value2 = split_values
                        bidno.append(value1.strip())
                    else:
                        pass
    return bidno

##### 2.3 개찰결과 크롤링

In [ ]:
def result_crawler(bidno):

    bid_list = []
    result_list = []
    pass_list = [] # 유찰된 데이터 개수 확인용

    for index, bid in enumerate(bidno):
        detail_url = f'https://www.g2b.go.kr:8101/ep/result/serviceBidResultDtl.do?bidno={bid}&bidseq=00&whereAreYouFrom=piser' # 개찰결과 상세조회 url
        r = requests.get(detail_url)
        soup = bs(r.text, "html.parser")

        try:
            # 개찰공고 정보 추출
            bid_number = soup.find('th', string="입찰공고번호").find_next('td').get_text(strip=True)
            bid_name = soup.find('th', string="공고명").find_next('td').get_text(strip=True)
            bid_where = soup.find('th', string="수요기관").find_next('td').get_text(strip=True)
            bid_who = soup.find('th', string="집행관").find_next('td').get_text(strip=True)
            bid_when = soup.find('th', string="실제개찰일시").find_next('td').get_text(strip=True)
            bid_list.append([index,bid_number,bid_name,bid_where,bid_who,bid_when])

            # 개찰결과 목록 추출
            rows = soup.find_all('tr') # 결과 테이블 데이터
            for row in rows:
                row_data = [cell.get_text(strip=True) for cell in row.find_all('td')]

                if len(row_data) >= 9: # 개찰 순위 데이터만 추가
                    row_data.insert(0, index) # 리스트 맨 앞에 인덱스(개찰공고 인덱스) 추가
                    result_list.append(row_data)

        except Exception as e: # 투찰한 모든 업체가 낙찰하한선 미달일 경우 예외처리
            pass_list.append(bid)
    
    return bid_list, result_list, pass_list

#### 3. 크롤링 함수 실행

In [ ]:
# 개찰공고 식별번호 추출
bidno = nara_crawler('QUERY', start_date, end_date)

# 개찰결과 크롤링
bid_list, result_list, pass_list = result_crawler(bidno)

#### 4. 데이터 저장

In [ ]:
# 개찰공고 데이터 
bid_df = pd.DataFrame(bid_list)
bid_df.columns = ['Index', '입찰공고번호', '공고명', '수요기관', '집행관', '실제개찰일시']
bid_df.to_excel('개찰공고_목록.csv',index=False)

# 개찰결과 데이터
result_df = pd.DataFrame(result_list)
result_df.columns = ['Index', '순위', '사업자등록번호', '업체명', '대표자명', '입찰금액', '투찰률(%)','추첨번호','투찰일시','비고']
result_df.to_excel('개찰결과_result.csv',index=False)

-----------------

In [ ]:
# # 유찰된 개찰데이터 확인용
# pass_list